<a href="https://colab.research.google.com/github/javierandresm13/03MIAR---Algoritmos-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Javier Moreno  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1Bs48ZaPJ3Bnt2ugMPeJqvivE2H0CJcb2?usp=sharing <br>

Problema:

Organizar los horarios de partidos de una jornada de La Liga<br>


## Descripción del problema:

Desde la Liga de Fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada, diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.






                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

## Modelo
### ¿Cómo se representa el espacio de soluciones?
El espacio de soluciones consiste en asignar partidos a horarios disponibles, formando una permutación. Dado que hay más partidos que horarios, se considera un subconjunto igual al número de horarios. Una solución es una lista de partidos asignados, como [("Real Madrid", "Barcelona"), ("Celta", "Valencia"), ...]. El tamaño del espacio de búsqueda se calcula con permutaciones
P(n, k) = n! / (n-k)!, lo que puede hacer que encontrar la solución óptima sea difícil computacionalmente.

### ¿Cuál es la función objetivo?
La función objetivo es maximizar la audiencia total de todos los partidos. Se calcula de la siguiente manera:
Audiencia individual del partido: Se determina para cada partido en un horario específico, considerando la categoría de los equipos y el coeficiente del horario.
Audiencia total: Es la suma de las audiencias individuales de todos los partidos en la asignación.
Penalización por coincidencias: Se aplica una penalización si varios partidos están programados simultáneamente, lo que reduce la audiencia total.

### ¿Cómo se implementan las restricciones?
La restricción principal es que cada franja horaria solo se puede asignar a un partido. Esto se maneja implícitamente mediante el uso de itertools.permutations, que asegura que cada partido se asigne a una franja horaria única en una permutación dada.



## Paso 1: Definir los datos proporcionados
Primero, definimos los horarios, categorías de equipos, audiencia base y coeficientes de reducción:


In [ ]:
horarios = ["V20", "S12", "S16", "S18", "S20", "D12", "D16", "D18", "D20", "L20"]
categorias = {"A": 3, "B": 11, "C": 6}
audiencia_base = {
    ("A", "A"): 2.0,
    ("A", "B"): 1.3,
    ("A", "C"): 1.0,
    ("B", "B"): 0.9,
    ("B", "C"): 0.75,
    ("C", "C"): 0.47
}
coeficientes_horarios = {
    "V20": 0.4, "S12": 0.55, "S16": 0.7, "S18": 0.8, "S20": 1.0,
    "D12": 0.45, "D16": 0.75, "D18": 0.85, "D20": 1.0, "L20": 0.4
}
coeficientes_coincidencia = [0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80]


## Paso 2: Generar todos los partidos posibles
Creamos listas de equipos y generamos todas las combinaciones posibles de partidos:

In [ ]:
equipos_A = ["Real Madrid", "R. Sociedad", "Barcelona"]
equipos_B = ["Celta", "Valencia", "Athletic", "Villareal", "Alavés", "Levante", "Espanyol", "Sevilla", "Betis", "Atlético", "Getafe"]
equipos_C = ["Mallorca", "Eibar", "Leganés", "Osasuna", "Granada", "Valladolid"]

partidos = []
for equipo1 in equipos_A + equipos_B + equipos_C:
    for equipo2 in equipos_A + equipos_B + equipos_C:
        if equipo1 < equipo2:  # Evitar duplicados y partidos consigo mismo
            categoria1 = 'A' if equipo1 in equipos_A else 'B' if equipo1 in equipos_B else 'C'
            categoria2 = 'A' if equipo2 in equipos_A else 'B' if equipo2 in equipos_B else 'C'
            partidos.append((equipo1, equipo2, categoria1, categoria2))
print(len(partidos))

190


## Paso 3: Función para calcular la audiencia de un partido en un horario específico
Definimos una función para calcular la audiencia de un partido dado en un horario específico:



In [ ]:
def calcular_audiencia(partido, horario):
    equipo1, equipo2, categoria1, categoria2 = partido
    base_audiencia = audiencia_base[(categoria1, categoria2)]
    coef_horario = coeficientes_horarios[horario]
    return base_audiencia * coef_horario

## Paso 4: Función para calcular la penalización por coincidencias
Definimos una función para calcular la penalización por coincidencias de horarios:

In [ ]:
def penalizacion_coincidencias(horarios_asignados):
    coincidencias = len(horarios_asignados) - len(set(horarios_asignados))
    return coeficientes_coincidencia[coincidencias]

## Paso 5: Generar todas las combinaciones posibles de asignación de horarios a partidos
Finalmente, generamos todas las combinaciones posibles de asignación de horarios a partidos y calculamos la audiencia total para encontrar la mejor asignación:

In [ ]:
import itertools

mejor_audiencia_total = -1
mejor_asignacion = None

for asignacion in itertools.permutations(partidos[:len(horarios)], len(horarios)):
    audiencia_total = sum(calcular_audiencia(partido, horario) for partido, horario in zip(asignacion, horarios))
    penalizacion = penalizacion_coincidencias(horarios)
    audiencia_total *= (1 - penalizacion)

    if audiencia_total > mejor_audiencia_total:
        mejor_audiencia_total = audiencia_total
        mejor_asignacion = asignacion

# Mostrar la mejor asignación encontrada
for partido, horario in zip(mejor_asignacion, horarios):
    print(f"Partido: {partido[0]} vs {partido[1]} - Horario: {horario}")

print(f"Audiencia total máxima: {mejor_audiencia_total}")

Partido: Real Madrid vs Valladolid - Horario: V20
Partido: Real Madrid vs Valencia - Horario: S12
Partido: Real Madrid vs Villareal - Horario: S16
Partido: Real Madrid vs Sevilla - Horario: S18
Partido: R. Sociedad vs Real Madrid - Horario: S20
Partido: R. Sociedad vs Valencia - Horario: D12
Partido: R. Sociedad vs Villareal - Horario: D16
Partido: R. Sociedad vs Sevilla - Horario: D18
Partido: Barcelona vs Real Madrid - Horario: D20
Partido: R. Sociedad vs Valladolid - Horario: L20
Audiencia total máxima: 10.13


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Análisis de Complejidad

Orden de Complejidad:
El problema de programar partidos para maximizar la audiencia es una tarea de optimización combinatoria. Implica explorar permutaciones de partidos y horarios. La complejidad está determinada por el número de permutaciones posibles, calculado como:

$$
P(n, k) = \frac{n!}{(n-k)!}
$$

donde $$n$$es el número total de partidos y $$k$$ es el número de franjas horarias. Este problema tiene una complejidad factorial $$ O  \frac{n!} {( n − k ) !}$$ que aumenta rápidamente, indicando su naturaleza NP-hard.

Implicaciones:
La complejidad del problema resalta la necesidad de técnicas que exploren eficientemente el espacio de

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

La técnica principal utilizada en el código es la generación de permutaciones con itertools.permutations, que permite una exploración exhaustiva de combinaciones de partidos y horarios. Esto maximiza las posibilidades de encontrar la mejor asignación debido a su cobertura amplia de soluciones potenciales. La función es sencilla de implementar y adecuada para el problema, ya que representa naturalmente la disposición de partidos en horarios limitados. Sin embargo, esta técnica puede ser computacionalmente costosa para problemas de gran tamaño, donde se podrían utilizar técnicas más avanzadas para mejorar la eficiencia. En resumen, es eficaz, con escalabilidad limitada.